# Técnicas e algoritmos de ataques ao RSA

### Funções Auxiliares

Vamos importar alguns pacotes para agilizar as operações aritmétricas dos algoritmos e para exibir as informações dos métodos.

In [296]:
from Crypto.Util.number import getPrime, long_to_bytes
from sympy import isprime, primerange
from typing import Callable
import time
from tqdm import tqdm
import random 
from math import gcd
from gmpy2 import (
    mpz, mpz_urandomb, next_prime, is_prime, lcm, powmod, fac
)

with open('words_pt.txt', 'r', encoding='utf-8') as file:
    words = [line.strip() for line in file if line.strip()]

In [ ]:
## Funções Aritmétricas
def isqrt(n):
    """
    Calcula a raiz quadrada inteira de n usando o método de Newton.
    
    Parâmetros:
    n (int): O número cujo valor inteiro da raiz quadrada será calculado.

    Retorna:
    int: A raiz quadrada inteira de n.
    """
    x = n
    y = (x + n // x) // 2
    while y < x:
        x = y
        y = (x + n // x) // 2
    return x

def is_perfect_square(n):
    '''
    If n is a perfect square it returns sqrt(n),
    
    otherwise returns -1
    '''
    h = n & 0xF; #last hexadecimal "digit"
    
    if h > 9:
        return -1 # return immediately in 6 cases out of 16.

    # Take advantage of Boolean short-circuit evaluation
    if (h != 2 and h != 3 and h != 5 and h != 6 and h != 7 and h != 8):
        # take square root if you must
        t = isqrt(n)
        if t*t == n:
            return t
        else:
            return -1
    
    return -1

def totiente(p: int, q: int):
    '''
    Função totiente em pq
    '''
    return (p-1)*(q-1)

def egcd(a: int, b: int): 
    '''
    Algoritmo Extendido de Euclides
    Retorna x, y, mdc(a,b) tal que ax + by = mdc(a,b)
    '''
    u, u1 = 1, 0
    v, v1 = 0, 1
    while b:
        q = a // b
        u, u1 = u1, u - q * u1
        v, v1 = v1, v - q * v1
        a, b = b, a - q * b
    return u, v, a

def mod_inverse(e: int, n: int):
    '''
    d tal que: de = 1 (mod n)
    Assumimos que é verdade que e é coprimo a n
    '''
    return egcd(e,n)[0]%n

def generate_e(p,q):
    phi_n = totiente(p,q)
    e = random.randint(2, phi_n - 1)
    while gcd(e, phi_n) != 1:
        e = random.randint(2, phi_n - 1)
    return e

def get_d_from_primes(e,p,q):
    return mod_inverse(e, totiente(p,q))

def get_smooth_prime(bits, smoothness=16):
    """
    Gera um primo "smooth", ou seja, um número primo que é próximo de
    um produto de fatores pequenos.

    Parâmetros:
    - state: Estado do gerador aleatório.
    - bits (int): Quantidade de bits desejada para o primo.
    - smoothness (int): Número de bits dos fatores pequenos.

    Retorna:
    - tuple: O primo gerado e seus fatores.
    """
    p = mpz(2)
    p_factors = [p]

    # Construir produto inicial de fatores pequenos
    while p.bit_length() < bits - 2 * smoothness:
        factor = getPrime(smoothness)
        p_factors.append(factor)
        p *= factor

    bitcnt = (bits - p.bit_length()) // 2

    # Ajustar até atingir o número desejado de bits
    while True:
        prime1 = getPrime(bitcnt)
        prime2 = getPrime(bitcnt)
        candidate = p * prime1 * prime2
        if candidate.bit_length() < bits:
            bitcnt += 1
            continue
        if candidate.bit_length() > bits:
            bitcnt -= 1
            continue
        if is_prime(candidate + 1):
            p_factors.append(prime1)
            p_factors.append(prime2)
            p = candidate + 1
            break

    p_factors.sort()
    return p, p_factors

def generate_bounded_prime(lower_bound, upper_bound): 
    # Gera um número primo aleatório definido por um limite inferior e superior
    if lower_bound > upper_bound: 
        raise ValueError("Lower bound must be less than or equal to upper bound.") 
     
    while True: 
        candidate = random.randint(lower_bound, upper_bound)  
        # Verifica se é primo
        if isprime(candidate): 
            return candidate 

In [345]:
ch = 120
def GenerateKeys(small_d=None, small_e=None, close_primes=None, n_bits=1024, smooth_primes=False):
    assert n_bits % 4 == 0
    if small_d == True:
        p = getPrime(n_bits)
        q = generate_bounded_prime(p+1, 2*p)
        N = p*q
        phi = totiente(p, q)
        # gera d tal que:
        #     mdc(d,phi(n)) = 1
        #    36d^4 < n
        good_d = False
        while not good_d:
            d = random.getrandbits(n_bits//4)
            if (gcd(d,phi) == 1 and 36*pow(d,4) < N):
                good_d = True
                        
        e = mod_inverse(d,phi)
    else:
        if close_primes == True:
            p = getPrime(n_bits)
            q = getPrime(n_bits-5)
            e = generate_e(p,q)

        elif small_e == True:
            p = getPrime(n_bits)
            q = getPrime(n_bits) 
            e = 3
            while gcd(e,(p-1)*(q-1))!= 1:
                p = getPrime(n_bits)
                q = getPrime(n_bits)

        elif smooth_primes == True:
                e = 0x10001
                while True:
                    p, p_factors = get_smooth_prime(n_bits, 16)
                    if len(p_factors) != len(set(p_factors)):
                        continue

                    q, q_factors = get_smooth_prime(n_bits, 17)
                    if len(q_factors) != len(set(q_factors)):
                        continue

                    factors = p_factors + q_factors
                    if e not in factors:
                        break

        else: 
            raise ValueError("Error: Method Not Found")

        d = get_d_from_primes(e,p,q)
        N = p*q

    print("="*ch)
    if n_bits >= 512:
        print(f"Chave Privada Gerada: (p = {p}\nq = {q}\nd = {d})\n")
        print(f"Chave Pública Gerada: (N = {N}\ne = {e})")
    else:
        print(f"Chave Privada Gerada: (p = {p}, q = {q}, d = {d})")
        print(f"Chave Pública Gerada: (N = {N}, e = {e})")
    print("="*ch)
    return p,q,d,N,e

def ApplyMethod(function: Callable[[int, int], tuple], N: int, e: int):
    start_time = time.time()  
    p,q,d = function(N,e)
    print("="*ch)
    print(f"Chave privada encontrada: (p = {p}, q = {q}, d = {d})")
    print(f"Tempo necessário: {time.time() - start_time} segundos.")
    print("="*ch)
    return p,q,d

def GenerateMessage(word_count):
    message = ' '.join(random.choice(words) for _ in range(word_count))
    print(f"Mensagem Enviada: {message}")
    print("="*ch)
    return message

    
def CriptMessage(message: str, N: int, e:int):
    max_block_size = (N.bit_length() - 1) // 8 

    message = message.encode('utf-8')
    blocks = [message[i:i + max_block_size] for i in range(0, len(message), max_block_size)]

    # Criptografa cada bloco individualmente
    ciphertext_blocks = []
    for block in blocks:
        int_ = int.from_bytes(block, byteorder='big')  # Converte o bloco para inteiro
        c = pow(int_, e, N)  # Criptografa o bloco
        ciphertext_blocks.append(c)
    return ciphertext_blocks

def DecryptMessage(ciphertext_blocks, d, n):
        """
        Descriptografa uma lista de blocos criptografados.
        
        Args:
            ciphertext_blocks (list): Lista de blocos criptografados como inteiros.
        
        Returns:
            str: Mensagem descriptografada como string.
        """

        # Descriptografa cada bloco
        plaintext_bytes = b""
        for block in ciphertext_blocks:
            plaintext_int = pow(block, d, n)  # Descriptografa o bloco
            block_bytes = plaintext_int.to_bytes((plaintext_int.bit_length() + 7) // 8, byteorder='big')
            plaintext_bytes += block_bytes  # Junta os bytes de cada bloco

        message = plaintext_bytes.decode('utf-8')  # Converte para string
        print(f"Mensagem Decifrada: {message}")
        print("="*ch)
        return message

def IntToMessage(self, ciphertext_blocks):
        """
        Descriptografa uma lista de blocos criptografados.
        
        Args:
            ciphertext_blocks (list): Lista de blocos criptografados como inteiros.
        
        Returns:
            str: Mensagem descriptografada como string.
        """
        d, n = self.private_key

        # Descriptografa cada bloco
        plaintext_bytes = b""
        for block in ciphertext_blocks:
            plaintext_int = pow(block, d, n)  # Descriptografa o bloco
            block_bytes = plaintext_int.to_bytes((plaintext_int.bit_length() + 7) // 8, byteorder='big')
            plaintext_bytes += block_bytes  # Junta os bytes de cada bloco

        return plaintext_bytes.decode('utf-8')  # Converte para string

## Força Bruta 

- **Descrição**: Realiza fatoração de um número $ N $ identificando seus divisores por meio de tentativa e erro. Testa sequencialmente possíveis divisores até encontrar fatores primos $p $ e $ q $.  
- **Ponto Forte**: Funciona bem para números pequenos ou quando $p $ e $ q $ são próximos.  
- **Ponto Fraco**: É ineficiente para números grandes, já que o tempo de execução cresce rapidamente com o tamanho de \( N \).  
- **Passo a Passo**:  
    1. Testa divisores sequencialmente a partir de  2.  
    2. Verifica se $ N $ é divisível por cada divisor $ t $.  
    3. Se $N \% t = 0$, retorna $ t $ e $ N/t $.  
- **Tempo de Execução**: $ O(\sqrt{N}) $ no pior caso, onde $ N $ é primo.  
- **Referências**:  
    - Rivest, R. L., Shamir, A., \& Adleman, L. (1978). *A Method for Obtaining Digital Signatures and Public-Key Cryptosystems*.


In [331]:
def força_bruta(N, e):
    """
    Realiza fatoração de um número N que é igual à multiplicação de dois primos, 
    usando busca por divisores. 

    Este método funciona bem para números com fatores relativamente pequenos.

    Exemplo: 
        N = 1039 * 58484513 = 60765409007, e = 7.
    
    Retorna:
        (p, q, d): Dois fatores de N, onde p * q = N. 

    Caso N seja primo, retorna:
        (N, 1, d)
    """

    # Verifica se N é divisível por 2 (paridade)
    if N % 2 == 0:
        d = get_d_from_primes(e, 2, N // 2)
        return 2, N // 2, d

    # Define o limite da busca como a raiz quadrada de N
    limite = isqrt(N)

    # Itera apenas sobre números ímpares a partir de 3 até a raiz quadrada de N
    for t in range(3, limite + 1, 2):
        if N % t == 0:  # Verifica se t é um divisor de N
            d = get_d_from_primes(e, t, N // t)
            return t, N // t , d

    # Caso nenhum divisor seja encontrado, N é primo
    d = get_d_from_primes(e, N, 1)
    return N, 1 , d

### Exemplo:

In [340]:
p, q, d, N, e = GenerateKeys(small_e=True, n_bits=12)
message = GenerateMessage(10)
ciphertext_blocks = CriptMessage(message, N, e)

Chave Privada Gerada: (p = 3467, q = 3881, d = 8965387)
Chave Pública Gerada: (N = 13455427, e = 3)
Mensagem Enviada: quer seria nada o atrás estivestes sob posição qual certeza


In [341]:
p, q, d = ApplyMethod(força_bruta, N, e)
message = DecryptMessage(ciphertext_blocks, d, N)

Chave privada encontrada: (p = 3467, q = 3881, d = 8965387)
Tempo necessário: 0.0002665519714355469 segundos.
Mensagem Decifrada: quer seria nada o atrás estivestes sob posição qual certeza


## Método de Fatoração de Fermat

- **Descrição**: Baseia-se na ideia de que qualquer número $ n $ ímpar pode ser expresso como $ n = a^2 - b^2 $. O objetivo é encontrar valores de $a $ e $ b $ que satisfaçam essa relação.  
- **Ponto Forte**: Muito eficiente quando  $p $ e $ q$ são próximos, ou seja, $ |p - q|$ é pequeno.  
- **Ponto Fraco**: Desempenho ruim quando $ p $ e $ q$ são muito distantes.  
- **Passo a Passo**:  
    1. Inicia com $a = \lceil \sqrt{N} \rceil $.  
    2. Calcula $b^2 = a^2 - N $.  
    3. Verifica se $ b^2 $ é um quadrado perfeito.  
    4. Caso seja, retorna $(a - b) $ e $ (a + b)$.  
- **Referências**:  
    - Antunes, Cristiane Medina. (2002). *Métodos de Fatoração de Números Inteiros*, disponível em (https://lume.ufrgs.br/bitstream/handle/10183/1626/000353932.pdf).

In [342]:
def fermat_factorization(N, e):
    """
    Realiza a fatoração de Fermat em um número composto N.

    Assumimos que N é um produto de dois primos ímpares próximos, p e q,
    tal que N = p * q e p ≈ q.

    Parâmetros:
    N (int): O número a ser fatorado.
    e (int): O expoente público do RSA.

    Retorna:
    tuple: Os fatores primos p e q de N e a chave privada d.

    Lança:
    AssertionError: Se N não for igual ao produto dos fatores encontrados.
    """
    # Caso trivial: N é par
    if N % 2 == 0:
        p, q = 2, N // 2
        d = get_d_from_primes(e, p, q)  # Calcula d usando p e q
        return p, q, d
    
    # Inicia com x como a raiz quadrada inteira de N
    x = isqrt(N)

    # Verifica se x já é um divisor direto de N
    if N % x == 0:
        p, q = x, N // x
        d = get_d_from_primes(e, p, q)  # Calcula d usando p e q
        return p, q, d

    # Itera para encontrar a decomposição N = a² - b²
    for i in range(x + 1, 1 + (N + 1) // 2):
        val = i**2 - N  # Calcula b² = a² - N
        y = isqrt(val)  # Calcula y como a raiz quadrada de b²

        # Verifica se val é um quadrado perfeito
        if y**2 == val:
            p, q = i - y, i + y
            d = get_d_from_primes(e, p, q)  # Calcula d usando p e q
            return p, q, d  # Retorna os fatores (p, q) e a chave privada d

    # Se nenhum fator for encontrado, N é primo
    d = get_d_from_primes(e, N, 1)  # Calcula d para o caso de N ser primo
    return N, 1, d


### Exemplo

In [346]:
p,q,d,N,e = GenerateKeys(close_primes=True, n_bits=20)
mensagem_original = GenerateMessage(15) # Gera uma frase de 15 polavras
mensagem_criptografada = CriptMessage(mensagem_original, N, e)

Chave Privada Gerada: (p = 812047, q = 16553, d = 3265713319)
Chave Pública Gerada: (N = 13441813991, e = 12285654439)
Mensagem Enviada: estou à dois sétimo primeira vossas estes nos desse daquela nossas mais fazeis tanto tive


In [347]:
p,q,d = ApplyMethod(fermat_factorization, N, e)
message = DecryptMessage(mensagem_criptografada, d, N)

Chave privada encontrada: (p = 16553, q = 812047, d = 3265713319)
Tempo necessário: 1.145678997039795 segundos.
Mensagem Decifrada: estou à dois sétimo primeira vossas estes nos desse daquela nossas mais fazeis tanto tive


## Realizando o ataque com o expoente $(e)$ pequeno

- **Descrição**: Explora fraquezas na escolha de um expoente público pequeno ($ e $), geralmente para acelerar o tempo de cifragem.  
- **Ponto Forte**: Muito eficaz quando $ e $ é pequeno e a mensagem $ m $ cifrada não é suficientemente aleatória ou protegida.  
- **Ponto Fraco**: Não funciona se as mensagens são devidamente aleatorizadas com preenchimento.  
- **Passo a Passo**:  
    1. Verifica se $ m^e < N$.  
    2. Recupera $ m $ extraindo diretamente a raiz $ e$-ésima de $ c$.  
- **Referências**:  
    - Boneh, D., \& Durfee, G. (1999). *Cryptanalysis of RSA with Small Public Exponent*.  
    - *Low Exponent Attack* por Ams Ghimire, disponível em (https://amsghimire.medium.com/low-exponent-attack-511bf5d227fc).

In [348]:
from gmpy2 import iroot

def small_e_attack(c: int, e: int) -> int:
    """
    Realiza o ataque de expoente pequeno (small e attack) ao módulo RSA.
    Para um expoente pequeno e (e.g., e = 3),
    se a mensagem m elevada a e for menor que o módulo n, a cifra c pode ser
    revertida diretamente extraindo a raiz e-ésima de c.

    Parâmetros:
    c (int): O valor cifrado (ciphertext).
    e (int): O expoente público do RSA.

    Retorna:
    int: A mensagem decifrada.
    """
    return int(iroot(c, e)[0])

In [349]:
p,q,d,N,e = GenerateKeys(small_e=True)
mensagem_original = GenerateMessage(10) # Gera uma frase de 15 polavras
mensagem_original = int.from_bytes(mensagem_original.encode(), "big")
mensagem_criptografada = pow(mensagem_original,e, N)

Chave Privada Gerada: (p = 99697876079592342135081914489206121532986379525544778465498148323678110785372982437399007634420580806869905511531643755566285181641626585558448651296791624434531347157819556487796627987049267012286236740131456073032348298021593629236053400540719637837508240367086587054556576244877643313138389502367220496213
q = 143117330143393566924525428055174047163595361873527814174010663801628997718919320334051722526993625575932393682171249668278609954507728017789796376041698780147043003598722553238440694519123377846682675438445373754301026525154708537358753276056295435578765857067923513330734751670923474370188130283928128779217
d = 9512329230318771707114326837801167232703737592642507721616301982735610986123343513890123988441446176671662481391927422882533204023525547148992796242343295207884804558936195801549767977035403815871158489180789653080402946876589046868676160159804383030618211060383810661251344553942439869315508061218142356025452081185865719092209973656299524768877

In [350]:
decrypted = small_e_attack(mensagem_criptografada, e)
decrypted = decrypted.to_bytes((decrypted.bit_length() + 7) // 8, "big")
print(f"Mensagem Decifrada: {decrypted.decode()}")

Mensagem Decifrada: possível sistema fazes dezanove alguns três apenas dizer faço minhas


## Método de Pollard p-1


- **Descrição**: O método \( p-1 \) de Pollard é baseado no Pequeno Teorema de Fermat e foi proposto por J.M. Pollard em 1974. Esse método permite a fatoração de um número $ n $ usando um fator primo $ p $ de $ n $ tal que $p-1$ divide $ k!$, onde $ k $ é um número inteiro positivo não muito grande, e os primos que dividem $ p-1 $ são relativamente pequenos.
- **Ponto Forte**: Muito eficaz para números onde $ p-1 $ possui pequenos fatores primos.  
- **Ponto Fraco**: Ineficiente para números onde $ p-1 $ possui grandes fatores primos.  
- **Passo a Passo**:  
   1. **Escolha de um número base $ a $**  
   Comece escolhendo uma base $a $, que é um número inteiro pequeno, por exemplo, $ a = 2 $.

   2. **Escolha de um limite $B$**: O algoritmo também exige um número limite $B$, que é um valor máximo de primos $p$ a serem usados. O limite $B$ será a "faixa" dentro da qual os divisores possíveis são procurados. O valor de $B$ afeta o tempo de execução do algoritmo: quanto maior $B$, mais tempo pode levar, mas também pode ser mais eficiente na descoberta de fatores pequenos.

   3. **Cálculo de $ a^{k!} \mod n $**  
   Calcule $ a^{k!} \mod n $, onde $ k! $ é o fatorial de um número $ k $, que é o produto dos números primos até um limite $ B $.

   4. **Verificação de $\gcd(a^{k!} - 1, n) > 1 $**  
   Aplique o algoritmo de Euclides para calcular o maior divisor comum ($ \gcd $) entre $ a^{k!} - 1 $ e $ n $. Se o resultado for maior que 1, significa que encontramos um divisor não trivial de $ n $.


- **Referências**:  
    - Pollard, J. M. (1974). *A Monte Carlo Method for Factorization*.  
    - Antunes, Cristiane Medina. (2002). *Métodos de Fatoração de Números Inteiros*, disponível em (https://lume.ufrgs.br/bitstream/handle/10183/1626/000353932.pdf).


In [351]:
def pollards_p_minus_1(n, e=None, a=2, B=65535):
    """
    Realiza a fatoração de um número composto `n` usando o método de Pollard's p-1.
    Depois utiliza os fatores para decifrar uma mensagem RSA.

    Parâmetros:
    - n (int): O módulo RSA a ser fatorado.
    - e (int): O expoente público do RSA.
    - c (int): A mensagem cifrada (ciphertext).
    - a (int): A base inicial usada no método de Pollard. Default é 2.
    - B (int): O limite inicial para o cálculo de b! (fatorial). Default é 65535.

    Retorna:
    - str: A mensagem decifrada.
    """
    while True:
        # Calcula b! (fatorial) e tenta encontrar um divisor comum com n
        b = fac(B)
        tmp2 = pow(a, b, n) - 1  # a^(b!) mod n - 1
        mdc = gcd(tmp2, n)

        # Avalia o valor do gcd e ajusta o limite B, se necessário
        if mdc == 1:
            B += 1  # Aumenta o limite B para continuar
        elif mdc == n:
            B -= 1  # Diminui o limite B para ajustar
        else:
            # Encontramos um fator p
            p = mdc
            q = n // p
            assert p * q == n, "Erro na fatoração: p * q != n"

            # Calcula o expoente privado d
            if e is not None:
                d = get_d_from_primes(e,p,q)
                return p,q,d
            else:
                return p,q

### Exemplo:

In [352]:
# Calcular os valores de RSA
p,q,d,N,e = GenerateKeys(smooth_primes=True)
mensagem_original = GenerateMessage(15) # Gera uma frase de 15 polavras
mensagem_criptografada = CriptMessage(mensagem_original, N, e)

Chave Privada Gerada: (p = 134904192821591509828542902531927287697361289235194800177292153103338738868415102407255445284800365786007253385515286336327032722570553148172649529836158625858170430565756490527086688996275951275744079548881103315299484429365243899305221284883601860621492477030017441366527836465783875710568762289963532167419
q = 90333176911019242184192919017537620639033184683252819820185751664981705769479124171614038337961595283174520388416462638850811127643605687911315973560261249909146937500997169414322729019451466928222116872437484229599426633670881730968728584057856781349196765638169590981617124010626403267151201584582922242003
d = 8780542602420905637817910609916051883547815962289489380005002752930943854144658824724028830126363365711856516615281538002137532224165181414773603442775987723172126971640866833816009225190685625585695026290500517757258807743285537396299124417755054991251707792608755738091648585417403695462667540898457180227152513356641346409584294181264534187415

In [353]:
p,q,d = ApplyMethod(pollards_p_minus_1, N, e)
message = DecryptMessage(mensagem_criptografada, d, N)

Chave privada encontrada: (p = 134904192821591509828542902531927287697361289235194800177292153103338738868415102407255445284800365786007253385515286336327032722570553148172649529836158625858170430565756490527086688996275951275744079548881103315299484429365243899305221284883601860621492477030017441366527836465783875710568762289963532167419, q = 90333176911019242184192919017537620639033184683252819820185751664981705769479124171614038337961595283174520388416462638850811127643605687911315973560261249909146937500997169414322729019451466928222116872437484229599426633670881730968728584057856781349196765638169590981617124010626403267151201584582922242003, d = 8780542602420905637817910609916051883547815962289489380005002752930943854144658824724028830126363365711856516615281538002137532224165181414773603442775987723172126971640866833816009225190685625585695026290500517757258807743285537396299124417755054991251707792608755738091648585417403695462667540898457180227152513356641346409584294181264534

## Ataque de Wiener

- **Descrição**: Como a exponenciação modular $m = c^d (mod \space N)$ leva tempo $O(log \space d)$,
para reduzir o tempo de descriptografia, pode-se desejar usar um valor relativamente pequeno de d.
No entanto, em 1991, Wiener mostrou que se o comprimento de bits de d for aproximadamente
um quarto daquele do módulo N, então é possível determinar o expoente privado
d da chave pública (N,e), ​​portanto, haveria uma quebra do criptosistema.
 Apesar disso, ao usar o Teorema de Legendre clássico em frações contínuas, em 1999 Boneh forneceu a primeira prova rigorosa que mostrou que o ataque de Wiener funciona na verdade, para $d < N^{0,25}/3$.
  
- **Ponto Forte**: Funciona bem quando $  d < N^{1/4}/3 $ .  
- **Ponto Fraco**: Ineficaz quando $ d $  é suficientemente grande.  
- **Passo a Passo**:  
    1. Criar uma estimativa próxima do expoente privado $d$ usando a chanve pública $(n,e)$, fazendo uso de sua expanção em frações contínuas.
    2. Verifica cada aproximação para determinar se $  d $  satisfaz as propriedades do RSA.  
- **Referências**:  
    - Israel e Vilmar, *Um estudo sobre o ataque de Wiener ao RSA* https://lume.ufrgs.br/bitstream/handle/10183/36090/000694121.pdf?sequence=1
    - Wiener, M. J. (1990). *Cryptanalysis of Short RSA Secret Exponents*.  
    - Inês Barbedo. *O Sistema Criptográfico RSA: Ataques e Variantes* 

### Teorema de Wiener

Seja $n=p q$, com $q<p<2 q$. Seja $d<\frac{1}{3} \sqrt[4]{n}$. \
Dado $n$ e $e$ com $e d \equiv 1 \bmod \phi(n)$, o atacante pode recuperar $d$ eficientemente.

In [318]:
def rational_to_contfrac(x: int, y: int) -> tuple[list[int], list[tuple[int, int]]]:
    """
    Converts a rational x/y fraction into a list of partial coefficients [a0, ..., an], and
    a list of convergents at each coefficient level [(n0, d0), (n1, d1), ...]

    O algortimo está disponível na seção 9.1 de https://r-knott.surrey.ac.uk/Fibonacci/cfINTRO.html#CFtofract

    Parâmetros:
        x (int): numerator of the given rational number
        y (int): denominator of the given rational number

    Retorna:
        tupla: a tuple of coefficients and convergents at each
        coefficient level
    """
    a = x // y
    cflist = [a]
    cvlist = [(a, 1)]
    ppn, ppd = 1, 0  # pre-pre numerator and denominator of convergent
    pn, pd = a, 1  # pre numerator and denominator of convergent
    while a * y != x:
        x, y = y, x - a * y
        a = x // y
        cflist.append(a)
        cn, cd = a * pn + ppn, a * pd + ppd
        cvlist.append((cn, cd))
        ppn, ppd = pn, pd
        pn, pd = cn, cd
    return cflist, cvlist


def wiener_attack(e: int, n: int) -> int:
    '''
    Finds d knowing (e,n)
    applying the Wiener continued fraction attack
    '''
    _, convergents = rational_to_contfrac(e, n)
    
    for (k,d) in convergents:
        #check if d is actually the key
        if k!=0 and (e*d-1)%k == 0:
            phi = (e*d-1)//k
            s = n - phi + 1
            # check if the equation x^2 - s*x + n = 0 has integer roots
            discr = s*s - 4*n
            if(discr>=0):
                t = is_perfect_square(discr)
                if t!=-1 and (s+t)%2==0:
                    return d

### Exemplo

In [399]:
p,q,d,N,e = GenerateKeys(small_d=True, n_bits=1024)
mensagem_original = GenerateMessage(15) # Gera uma frase de 15 polavras
mensagem_criptografada = CriptMessage(mensagem_original, N, e)

Chave Privada Gerada: (p = 100898549801303746894175774793427430226832514882456826867366510885695793749720438171793712729181571187187119224590737124547963126522891465064400744252700887881633455802450010973132841739183822948218999761427505381842900869209441931538779396451886950641510047974116108652150727284908047460149899141109666854667
q = 177493404796774743528875026897784428710028182303024255450527783908338082527335899742222690290644282595376641374437818027085908896474866037171013205350468972479485355976843407413744799694369209698414214884277342207349984575422596653512758160847060189883946677800487064339899352638968391775914836033184882985759
d = 21346556897417574221673527938469190048616149829329166381778304402907793330073)

Chave Pública Gerada: (N = 1790882714329034181457090327907874343938045812702215069458188551823947370062789022755739957844570713457002729913189619600390716299247110204082341909416408118672023269198945214186964798590639193827651586429857800250214488539046307941791123

In [402]:
d = wiener_attack(e,N)
print("Expoente Descriptografador: d =", d)
message = DecryptMessage(mensagem_criptografada, d, N)

Expoente Descriptografador: d = 21346556897417574221673527938469190048616149829329166381778304402907793330073
Mensagem Decifrada: minhas zero tem para adeus posso puderam vossa grupo eles viagem à vais quanto apoio


# Interface 

In [405]:
import time
from math import gcd, isqrt
import emoji


class SimuladorAtaqueRSA:
    def __init__(self, n, e=None, c=None, m=None, d=None):
        self.n = n
        self.e = e
        self.c = c
        self.m = m
        self.d = d

    def fatoracao_fermat(self):
        print(f"\n🔍 Executando Fatoração de Fermat para N={self.n}")
        p, q, d = fermat_factorization(self.n, self.e)
        print(f"🎯 Os fatores primos de {self.n} são:\n p = {p}\n q = {q}")
        print(f"🔑 A chave privada é: d = {d}")
        message = DecryptMessage(self.c, d, self.n)
        print(f"🔓 Mensagem decifrada: {message}")

        return
       

    def pollard_p_menos_1(self, B=10**6):
        # print(f"\n⚡ Executando Método Pollard p-1")
        message = pollards_p_minus_1(n, e, c)
        print(f"Mensagem decifrada: {message}")
        

def menu_principal():
    print("""
  ██████╗ ███████╗ █████╗     █████╗ ████████╗████████╗ █████╗  ██████╗██╗  ██╗
  ██╔═██║ ██╔════╝██╔══██╗   ██╔══██╗╚══██╔══╝╚══██╔══╝██╔══██╗██╔════╝██║ ██║
  ██║ ██║ ██║     ██║  ██╗   ██║  ██╗   ██║      ██║   ██║  ██╗██║     ██║ ██║
  ████║   ███████╗███████║   ███████║   ██║      ██║   ███████║██║     ████║
  ██║ ██╗      ██║██╔══██║   ██╔══██║   ██║      ██║   ██╔══██║██║     ██╔═██║
  ██║  ██║███████║██║  ██║   ██║  ██║   ██║      ██║   ██║  ██║╚██████╗██║  ██║
  ╚═╝  ╚═╝╚══════╝╚═╝  ╚═╝   ╚═╝  ╚═╝   ╚═╝      ╚═╝   ╚═╝  ╚═╝ ╚═════╝╚═╝  ╚═╝

                                               .----.
                                .------------. | == |
                                |.-========-.| |----|
                                ||          || | == |
                                ||          || |----|
                                |'-..... ..-'| |::::|
                                  `"")---(""`  |___.|
                                /::::::::::::\"__  "
                               /::::=======:::\  \`\ 
              <!--    __________   __  ______  ______       -->
              <!--   / __/ ___| | / / / __/  |/  / _ | ___  -->
              <!--  / _// (_ /| |/ / / _// /|_/ / __ |/ _ \ -->
              <!-- /_/  \___/ |___/ /___/_/  /_/_/ |_/ .__/ -->
              <!--                                  /_/     -->  

12537508673247452740415065632494689029027732950928380086256250024867187926128380
80287907219466080126910572928733538110840068106336709993591705426997197501264265
37797307222843941099150274102793206236042838704774026115105372038660088467920614
59271374450331751970488423409043692985323238050322405992071593773700020204905132
60850861669417224690840035708894072063281961794947745531844449231173387949796417
48472790731929346064345414201887091548869282344794136434276044068819968354034453
81296531655214965307063040023445866227631412406489130525362013420587466334224983
906533723001807034527677883322082180058876687814462664149 %¨$$#¨&*(()((¨#@$&_)))
               

""")

    print("🤖 Bem-vindo ao Simulador de Ataques RSA!")
    escolha = int(input("""Escolha o tipo de ataque:\n
          [1] Fatoração de Fermat \n
          [2] Método de Pollard p-1\n
                        
          Sua escolha: """))
    



    t_inicio = time.perf_counter()
    if escolha == 1:
        print("🔑 Por favor, insira os dados necessários para o ataque:")
        c = int(input("🔐 Digite a mensagem para realizar o ataque: "))
        n = int(input("🔢 Digite o número a ser fatorado,, chave pública (N) : "))
        e = int(input("🔑 Digite o valor de expoente da chave pública (e): ") )
        print("✅ Dados recebidos com sucesso!", n , e)
        ataque = SimuladorAtaqueRSA(n,e,c)
        resultado = ataque.fatoracao_fermat()
    elif escolha == 2:
        print(" Por favor, insira os dados necessários para o ataque:")
        n = int(input("🔑 Digite o valor de n (chave pública): ") )
        e = int(input("🔑 Digite o valor de e (chave pública): ") )
        c = int(input("🔐 Digite o valor de c (valor cifrado): "))
        print("✅ Dados recebidos com sucesso!")
        ataque = SimuladorAtaqueRSA(n,e,c)
        mensagens = [
        "[*] Hackeando sistemas...",
        "[*] Acesso autorizado!",
        "[*] Preparando ataque...",
        "[*] RSA Attack: Ativado!",
        "[*] Ataque completo!"
        ]
            
        time.sleep(1)

        print(f"\n⚡ Executando o Método Pollard p-1")

        for msg in mensagens:
            print(msg)
            time.sleep(2)  # Pausa entre as mensagens
        resultado = ataque.pollard_p_menos_1()

    else:
        print("⚠️ Escolha inválida.")
        return
    t_fim = time.perf_counter()

    # print("\n🎯 Resultado do ataque:")
    # print(f"{resultado}")
    print(f"⏱️ Tempo de execução: {t_fim - t_inicio:.2f} segundos")

# if __name__ == "__main__":
#     menu_principal()

# Referências:

- https://braghetto.eti.br/files/trabalho%20oficial%20final%20rsa.pdf
- https://www.iacr.org/archive/crypto2007/46220388/46220388.pdf